<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/TweepyPush2Kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tweepy
!pip install kafka-python

In [0]:
from kafka import KafkaProducer, KafkaConsumer
from kafka.errors import KafkaError
from json import dumps, loads

import tweepy

In [0]:
producer = KafkaProducer(bootstrap_servers=['18.216.34.97:9092'],
                         value_serializer=lambda x: dumps(x).encode('utf-8'))

### Setup Streaming

In [0]:
class LDSStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status.text)
        # modify topic here
        producer.send('tweepyv1', value=status._json) 

In [0]:
consumer_key  = "2wrYTQnh4poBbF5MsubR3sk4k"
consumer_secret = "93Yi0isNNgNPW4bFo8hsMpANUIw61lofgRRpz8H39P5V6o63tl"
access_token  = "2761651876-LoLkgE3EFYkqIHSGEPvzB3udtmATj4XNem2MemW"
access_token_secret = "1uDlUpdk6mrinFKFl3vN06xIVeORaa60EjubXrEOSpYKc"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [0]:
myStream = tweepy.Stream(auth=api.auth, listener=LDSStreamListener())

### Start Streaming

In [0]:
myStream.filter(track=['avengers'], languages=['en'])

### Consumer Test

In [0]:
consumer = KafkaConsumer(
    'tweepyv1',
     bootstrap_servers=['18.216.34.97:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group3102',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [0]:
for message in consumer:
    message = message.value
    print(message)